In [6]:
"""
BACnet Device Simulator with Environmental Sensors

This script creates a simulated BACnet device that mimics environmental monitoring
equipment with various sensors (temperature, humidity, pressure, frequency) and
heating control outputs. It continuously updates sensor values with random data
to simulate real-world fluctuations.

Dependencies:
- BAC0 library for BACnet communication
- time module for sleep functionality
- random module for generating simulated sensor values
"""

from time import sleep
import random
#from BAC0.core.devices.local.object import ObjectFactory
from bacpypes.object import ScheduleObject
from bacpypes.primitivedata import Real
from BAC0 import lite,connect
from BAC0.core.devices.local.models import (
analog_input,
analog_output,
binary_output,
binary_input,
)

def defining_objects(device):
    """
    Define and create BACnet objects for the simulated device.
    
    This function creates various BACnet objects including:
    - Analog inputs for environmental sensors
    - Binary outputs for heating control
    - Binary inputs for radiator state monitoring
    
    Args:
        device: BACnet device object to add the objects to
    
    Returns:
        Result of adding objects to the device application
    """
    
    # Define Analog Input Objects for Environmental Sensors
    
    # Frequency sensor (could represent AC power frequency or equipment frequency)
    analog_input(
        instance=10,
        name="Frequency",
        properties={"units": "hertz"},
        description="System frequency measurement",
        presentValue=50.0,  # Default to 50Hz (European standard)
    )

    # Barometric pressure sensor
    analog_input(
        instance=20,
        name="Barometer",
        properties={"units": "pascals"},
        description="Atmospheric pressure sensor",
        presentValue=101325.0,  # Standard atmospheric pressure
    )

    # Humidity sensor
    analog_input(
        instance=30,
        name="Humidity",
        properties={"units": "percent"},
        description="Relative humidity measurement",
        presentValue=45.0,  # Comfortable indoor humidity
    )

    # Temperature sensor
    analog_input(
        instance=40,
        name="Temperature",
        properties={"units": "degreesCelsius"},
        description="Ambient temperature sensor",
        presentValue=22.0,  # Comfortable room temperature
    )

    # Gas resistance sensor (for air quality monitoring)
    analog_input(
        instance=50,
        name="GasResistance",
        properties={"units": "ohms"},
        description="Gas sensor resistance for air quality",
        presentValue=50000.0,  # Typical gas sensor resistance
    )

    # Define Binary Output Objects for Heating Control
    
    # Room 1 heating enable control
    binary_output(
        instance=10,
        name="RoomOneHeatingEnabled",
        description="Enable/disable heating for room one",
        presentValue=False,  # Start with heating disabled
    )

    # Room 2 heating enable control
    binary_output(
        instance=20,
        name="RoomTwoHeatingEnabled",
        description="Enable/disable heating for room two",
        presentValue=False,  # Start with heating disabled
    )

    # Define Binary Input Objects for Status Monitoring
    
    # Room 1 radiator state feedback
    binary_input(
        instance=10,
        name="RoomOneRadiatorState",
        description="Current on/off state of room one radiator",
        presentValue=False,  # Radiator initially off
    )

    # Room 2 radiator state feedback
    binary_input(
        instance=20,
        name="RoomTwoRadiatorState",
        description="Current on/off state of room two radiator",
        presentValue=False,  # Radiator initially off
    )

def simulate_sensor_values(device):
    """
    Simulate realistic sensor value changes.
    
    This function updates the present values of various sensors with
    realistic random variations to simulate real-world conditions.
    
    Args:
        device: BACnet device object containing the sensors to update
    """
    
    # Simulate frequency variations (typically 49.5-50.5 Hz for European grid)
    device["Frequency"].presentValue = random.uniform(49.5, 50.5)
    
    # Simulate temperature variations (18-26°C indoor range)
    device["Temperature"].presentValue = random.uniform(18.0, 26.0)
    
    # Simulate humidity variations (30-70% comfortable range)
    device["Humidity"].presentValue = random.uniform(30.0, 70.0)
    
    # Simulate barometric pressure variations (typical range)
    device["Barometer"].presentValue = random.uniform(99000, 103000)
    
    # Simulate gas resistance variations (air quality changes)
    device["GasResistance"].presentValue = random.uniform(40000, 80000)

def main():
    """
    Main function to run the BACnet device simulator.
    
    This function:
    1. Establishes connection to BACnet network
    2. Defines all device objects
    3. Runs continuous simulation loop
    """
    
    try:
        # Connect to BACnet network
        # IP: 192.168.1.9 with subnet mask /24
        # Port: 47808 (standard BACnet port)
        # Device ID: 3056486 (unique identifier for this device)
        print("Connecting to BACnet network...")
        device1 = connect(ip="192.168.1.9/24", port=47808, deviceId=3056486)
        print("Connected successfully!")
        
        # Define all BACnet objects for this device
        print("Defining BACnet objects...")
        defining_objects(device1)
        print("Objects defined successfully!")
        
        print("Starting simulation loop...")
        print("=" * 50)
        
        # Main simulation loop
        while True:
            # Update all sensor values with simulated data
            simulate_sensor_values(device)
            
            # Display current sensor readings
            print(f"Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"Frequency: {device1['Frequency'].presentValue:.2f} Hz")
            print(f"Temperature: {device1['Temperature'].presentValue:.2f} °C")
            print(f"Humidity: {device1['Humidity'].presentValue:.2f} %")
            print(f"Barometer: {device1['Barometer'].presentValue:.2f} Pa")
            print(f"Gas Resistance: {device1['GasResistance'].presentValue:.2f} Ω")
            print(f"Room 1 Heating: {'ON' if device1['RoomOneHeatingEnabled'].presentValue else 'OFF'}")
            print(f"Room 2 Heating: {'ON' if device1['RoomTwoHeatingEnabled'].presentValue else 'OFF'}")
            print("-" * 50)
            
            # Wait 2 seconds before next update
            sleep(2)
            
    except KeyboardInterrupt:
        print("\nSimulation stopped by user.")
    except Exception as e:
        print(f"Error occurred: {e}")
        print("Please check network connection and BACnet configuration.")

# Additional utility functions for enhanced functionality

def toggle_heating(device, room_number):
    """
    Toggle heating on/off for a specific room.
    
    Args:
        device: BACnet device object
        room_number: Room number (1 or 2)
    """
    if room_number == 1:
        current_state = device["RoomOneHeatingEnabled"].presentValue
        device["RoomOneHeatingEnabled"].presentValue = not current_state
        print(f"Room 1 heating {'enabled' if not current_state else 'disabled'}")
    elif room_number == 2:
        current_state = device["RoomTwoHeatingEnabled"].presentValue
        device["RoomTwoHeatingEnabled"].presentValue = not current_state
        print(f"Room 2 heating {'enabled' if not current_state else 'disabled'}")

def get_device_status(device):
    """
    Get comprehensive status of all device objects.
    
    Args:
        device: BACnet device object
    
    Returns:
        Dictionary containing current status of all objects
    """
    status = {
        "sensors": {
            "frequency": device["Frequency"].presentValue,
            "temperature": device["Temperature"].presentValue,
            "humidity": device["Humidity"].presentValue,
            "barometer": device["Barometer"].presentValue,
            "gas_resistance": device["GasResistance"].presentValue,
        },
        "heating": {
            "room_1_enabled": device["RoomOneHeatingEnabled"].presentValue,
            "room_2_enabled": device["RoomTwoHeatingEnabled"].presentValue,
        },
        "radiators": {
            "room_1_state": device["RoomOneRadiatorState"].presentValue,
            "room_2_state": device["RoomTwoRadiatorState"].presentValue,
        }
    }
    return status

# Run the simulation if this script is executed directly
if __name__ == "__main__":
    import time  # Import time module for timestamp display
    main()

ModuleNotFoundError: No module named 'BAC0.core.devices.local.models'